In [1]:
%matplotlib widget
import matplotlib.pyplot as plt
import numpy as np
from scipy.ndimage import correlate
from skimage.filters import *
from scipy.stats import entropy
from LZWUtil import *


In [2]:
  # utility: what is arr?
def arrInfo(arr):
    return arr.shape, arr.min(), arr.max(), arr.dtype

In [3]:
I = plt.imread('im.jpg')
plt.figure()
arrInfo(I)
plt.imshow(I)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [4]:
bins = np.arange(256)/256
f, axarr = plt.subplots(1,2, figsize=(10, 3))
Ic = I/I.ravel().max()
axarr[0].imshow(I)
axarr[1].hist(Ic[...,0].ravel(), bins, alpha = .6, label = 'red', color = 'r')
axarr[1].hist(Ic[...,1].ravel(), bins, alpha = .6, label = 'green', color = 'g')
axarr[1].hist(Ic[...,2].ravel(), bins, alpha = .6, label = 'blue', color = 'b')
axarr[1].legend(loc = 'upper right')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [5]:
h = np.array([-1,1], ndmin = 2).astype(np.int16)
h1 = np.transpose(h)
h2 = np.stack((np.array([0,-1]), np.array([-1,2])))
def correl(fil):
    If_red = correlate(I[...,0].astype(np.int16), fil, mode='constant', cval=0)
    If_green = correlate(I[...,1].astype(np.int16), fil, mode='constant', cval=0)
    If_blue = correlate(I[...,2].astype(np.int16), fil, mode='constant', cval=0)
    return If_red, If_green, If_blue

In [6]:
If = np.zeros(I.shape, dtype = np.int16) # both sides display
# correlate
If_red, If_green, If_blue = correl(h)
# setting channels
If[...,0] = If_red
If[...,1] = If_green
If[...,2] = If_blue

# normalized
If_copy = If.copy()
If_copy = (If_copy - If_copy.min())/(If_copy.max() - If_copy.min())

In [7]:
If1 = np.zeros(I.shape, dtype = np.int16) # both sides display

# correlate
If1_red, If1_green, If1_blue = correl(h1)
If1[...,0] = If1_red
If1[...,1] = If1_green
If1[...,2] = If1_blue

# normalized
If_copy1 = If1.copy()
If_copy1 = (If_copy1 - If_copy1.min())/(If_copy1.max() - If_copy1.min())

In [8]:
If2 = np.zeros(I.shape, dtype = np.int16)

# correlate
If2_red, If2_green, If2_blue = correl(h2)
If2[...,0] = If2_red
If2[...,1] = If2_green
If2[...,2] = If2_blue

# normalized
If_copy2 = If2.copy()
If_copy2 = (If_copy2 - If_copy2.min())/(If_copy2.max() - If_copy2.min())

In [9]:
# plot
bins=np.arange(-255,256,1)
f, axar = plt.subplots(1,2, figsize=(10, 3))

axar[0].imshow(If_copy, cmap='gray', interpolation=None)
axar[0].set_title("Sub Left Prediction")
axar[1].hist(If[...,0].ravel(), bins, alpha = .6, label = 'red', color = 'r')
axar[1].hist(If[...,1].ravel(), bins, alpha = .6, label = 'green', color = 'g')
axar[1].hist(If[...,2].ravel(), bins, alpha = .6, label = 'blue', color = 'b')
axar[1].legend(loc = 'upper right')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [10]:
# plot
bins=np.arange(-255,256,1)
f, axar = plt.subplots(1,2, figsize=(10, 3))

axar[0].imshow(If_copy1, cmap='gray', interpolation=None)
axar[0].set_title("Up Prediction")
axar[1].hist(If1[...,0].ravel(), bins, alpha = .6, label = 'red', color = 'r')
axar[1].hist(If1[...,1].ravel(), bins, alpha = .6, label = 'green', color = 'g')
axar[1].hist(If1[...,2].ravel(), bins, alpha = .6, label = 'blue', color = 'b')
axar[1].legend(loc = 'upper right')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [11]:
# plot
bins=np.arange(-255,256,1)
f, a = plt.subplots(1,2, figsize=(10, 3))

a[0].imshow(If_copy2, cmap='gray', interpolation=None)
a[0].set_title("Avg Prediction")
a[1].hist(If2[...,0].ravel(), bins, alpha = .6, label = 'red', color = 'r')
a[1].hist(If2[...,1].ravel(), bins, alpha = .6, label = 'green', color = 'g')
a[1].hist(If2[...,2].ravel(), bins, alpha = .6, label = 'blue', color = 'b')
a[1].legend(loc = 'upper right')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [12]:
print(arrInfo(If))
arrInfo(If_copy1)

((560, 1000, 3), -152, 162, dtype('int16'))


((560, 1000, 3), 0.0, 1.0, dtype('float64'))

In [13]:
def printStatsChannel(origin, name):
    '''
    Prints the info of the encoding for each channel
    origin: the raveled color channel
    name: name of the channel
    '''
    # total entropy = - sum(bin 0 -> bin 255) of probability(event) * log2(probability(event))
    # then just compute the size
    # then the encoded file, comparing the bits/pixel :D, should be roughly the same with the entropy
    freq, bins = np.histogram(origin, bins = np.arange(257))
    print(name + " channel entropy is " + str(entropy(freq, base=2)))

In [14]:
printStatsChannel(I, "Origin")

Origin channel entropy is 7.288067143497486


In [15]:
printStatsChannel(If, "Sub Left")

Sub Left channel entropy is 3.749647372577736


In [16]:
printStatsChannel(If1, "Up")

Up channel entropy is 3.798689241247225


In [17]:
printStatsChannel(If2, "Average")

Average channel entropy is 4.448180860979805


In [18]:
arrInfo(If_copy)

((560, 1000, 3), 0.0, 1.0, dtype('float64'))

In [19]:
If_copy = (255*(If_copy)).astype('uint8')
If_copy1 = (255*(If_copy1)).astype('uint8')
If_copy2 = (255*(If_copy2)).astype('uint8')

In [20]:
BITS = 10
Ifd = If_copy.ravel()
estream = LZWCompress(Ifd, bits=BITS)
encodedSize = sum(1 for x in estream)
print('I size for sub left %6.3fKB, LZ encoded size %6.3fKB.' %
      (If_copy.size/1024, encodedSize*BITS/(8*1024)))

I size for sub left 1640.625KB, LZ encoded size 960.568KB.


In [22]:
Ifd1 = If_copy1.ravel()
estream1 = LZWCompress(Ifd1, bits=BITS)
encodedSize1 = sum(1 for x in estream)
print('I size for sub left %6.3fKB, LZ encoded size %6.3fKB.' %
      (If_copy1.size/1024, encodedSize1*BITS/(8*1024)))

I size for sub left 1640.625KB, LZ encoded size 960.568KB.


In [23]:
Ifd2 = If_copy2.ravel()
estream2 = LZWCompress(Ifd2, bits=BITS)
encodedSize2 = sum(1 for x in estream)
print('I size for sub left %6.3fKB, LZ encoded size %6.3fKB.' %
      (If_copy2.size/1024, encodedSize2*BITS/(8*1024)))

I size for sub left 1640.625KB, LZ encoded size 960.568KB.


In [27]:
decomp = LZWDecompress(estream1, bits=BITS)
recI1d = np.zeros(Ifd1.shape)
for i, x in enumerate(decomp):
    recI1d[i] = x

In [28]:
Ir = recI1d.reshape(I.shape)

In [32]:
f, ax = plt.subplots(1,2, figsize=(10,3), sharex=True, sharey=True)
ax[0].imshow(If_copy1, cmap='gray')
ax[0].set_title('Up Filter %6.3fKB' % (I.size/1024))

ax[1].imshow(Ir.astype('uint8'), cmap='gray')
ax[1].set_title('LZ Compressed %6.3fKB' % (encodedSize*BITS/(8*1024)))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 1.0, 'LZ Compressed 960.568KB')

In [34]:
printStatsChannel(Ir, "LZ Compressed")
printStatsChannel(If_copy1, "Up - Normalized")
printStatsChannel(If1, "Up")

LZ Compressed channel entropy is 4.3266100250176285
Up - Normalized channel entropy is 4.3266100250176285
Up channel entropy is 3.798689241247225


In [ ]:
plt.close('all')